# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 29 2022 8:47AM,247533,20,ALU0008034,Alumier Labs Inc.,Released
1,Sep 29 2022 8:47AM,247533,20,ALU0008039,Alumier Labs Inc.,Released
2,Sep 29 2022 8:46AM,247532,10,8587670-SD,Eywa Pharma Inc.,Released
3,Sep 29 2022 8:45AM,247530,18,TASA-50012903,TASA USA Inc.,Released
4,Sep 29 2022 8:42AM,247528,10,CTF0011434,"Citieffe, Inc.",Released
5,Sep 29 2022 8:39AM,247527,10,0086023318,ISDIN Corporation,Released
6,Sep 29 2022 8:39AM,247527,10,0086023322,ISDIN Corporation,Released
7,Sep 29 2022 8:39AM,247527,10,0086023708,ISDIN Corporation,Released
8,Sep 29 2022 8:39AM,247527,10,0086023714,ISDIN Corporation,Released
9,Sep 29 2022 8:39AM,247527,10,0086023734,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,247527,Released,16
15,247528,Released,1
16,247530,Released,1
17,247532,Released,1
18,247533,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247527,NaN,NaN,16.0
247528,NaN,NaN,1.0
247530,NaN,NaN,1.0
247532,NaN,NaN,1.0
247533,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247395,0.0,0.0,1.0
247471,0.0,0.0,1.0
247480,0.0,0.0,1.0
247489,0.0,0.0,1.0
247490,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247395,0,0,1
247471,0,0,1
247480,0,0,1
247489,0,0,1
247490,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247395,0,0,1
1,247471,0,0,1
2,247480,0,0,1
3,247489,0,0,1
4,247490,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247395,,,1
1,247471,,,1
2,247480,,,1
3,247489,,,1
4,247490,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.
2,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.
3,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.
4,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc."
5,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation
21,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.
22,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc."
23,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations"
24,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc."
27,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.,,,2
1,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.,,,1
2,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.,,,1
3,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc.",,,1
4,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation,,,16
5,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,,,1
6,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",,,1
7,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",,,1
8,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",,,3
9,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",,3,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.,2,,
1,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.,1,,
2,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.,1,,
3,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc.",1,,
4,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation,16,,
5,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,1,,
6,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",1,,
7,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",1,,
8,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",3,,
9,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",15,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.,2,,
1,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.,1,,
2,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.,1,,
3,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc.",1,,
4,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.,2,NaN,NaN
1,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.,1,NaN,NaN
2,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.,1,NaN,NaN
3,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc.",1,NaN,NaN
4,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation,16,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 8:47AM,247533,20,Alumier Labs Inc.,2,0.0,0.0
1,Sep 29 2022 8:46AM,247532,10,Eywa Pharma Inc.,1,0.0,0.0
2,Sep 29 2022 8:45AM,247530,18,TASA USA Inc.,1,0.0,0.0
3,Sep 29 2022 8:42AM,247528,10,"Citieffe, Inc.",1,0.0,0.0
4,Sep 29 2022 8:39AM,247527,10,ISDIN Corporation,16,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1485086,21,0.0,10.0
16,247480,1,0.0,0.0
18,247530,1,0.0,0.0
19,742575,19,3.0,0.0
20,495053,3,0.0,0.0
21,989870,4,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1485086,21,0.0,10.0
1,16,247480,1,0.0,0.0
2,18,247530,1,0.0,0.0
3,19,742575,19,3.0,0.0
4,20,495053,3,0.0,0.0
5,21,989870,4,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21,0.0,10.0
1,16,1,0.0,0.0
2,18,1,0.0,0.0
3,19,19,3.0,0.0
4,20,3,0.0,0.0
5,21,4,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,16,Released,1.0
2,18,Released,1.0
3,19,Released,19.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21
Status,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,3.0,0.0,0.0
Released,21.0,1.0,1.0,19.0,3.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,3.0,0.0,0.0
2,Released,21.0,1.0,1.0,19.0,3.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,3.0,0.0,0.0
2,Released,21.0,1.0,1.0,19.0,3.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()